

# Fast Causal inference

Fast Causal Inference is Tencent's first open-source causal inference project. It is an OLAP-based high-performance causal inference (statistical model) computing library, which solves the performance bottleneck of existing statistical model libraries (R/Python) under big data, and provides causal inference capabilities for massive data execution in seconds and sub-seconds. At the same time, the threshold for using statistical models is lowered through the SQL language, making it easy to use in production environments. At present, it has supported the causal analysis of WeChat-Search, WeChat-Video-Account and other businesses, greatly improving the work efficiency of data scientists.
![Example Image](https://github.com/Tencent/fast-causal-inference/raw/main/docs/images/fast-causal-inference3.png)


# Get started

In [ ]:
import time
import pandas as pd

In [ ]:
import fast_causal_inference
ais = fast_causal_inference.FCIProvider('all_in_sql')
df = ais.readClickHouse('test_data_small')

In [ ]:
df.show()

# know your data

### test data generation

In [ ]:
import numpy as np
import pandas as pd
import random
# Set random seed for reproducibility
np.random.seed(0)
n = 10000

################################## generate covariables #############################################
# Generate x1-x5, they come from different distributions and have different variances
x1 = np.random.normal(0, 1, n) # Normal distribution, variance is 1
x2 = np.random.normal(0, 2, n)  # Normal distribution, variance is 4
x3 = np.random.exponential(1, n)  # Exponential distribution, variance is 1
x4 = np.random.exponential(2, n)  # Exponential distribution, variance is 4
x5 = np.random.uniform(0, 5, n)  # Uniform distribution, variance is approximately 1.33
weight = np.random.uniform(0, 1, n) # use for sample reweighing

# Generate x6-x7, they are long-tail distributed data
x_long_tail1 = np.random.pareto(3, n)  # Pareto distribution
x_long_tail2 = np.random.pareto(2, n)  # Pareto distribution

# Generate x_cat1, it is a discrete variable of string type
x_cat1 = np.random.choice(['A', 'B', 'C', 'D', 'E'], n)
###################################################################################################### 



################################## generate exprimental data ######################################### 
# Generate treatment, it is a binary random variable
treatment = np.random.choice([0, 1], n)


# Generate y 
# y is highly correlated with x1, x2, x3,x_long_tail2 and treatment
# and there is heterogeneity on x_cat1 and x1,x2,x4
y_pre = x1 + 2*x2 + 3*x3 + 3*np.log(x_long_tail2+1) + np.random.normal(0, 1, n)
y = y_pre + 5*treatment + treatment*(x1+x2**2+np.log(x4+1)+(x_long_tail1>1))*2 + np.random.normal(0, 2, n)
y[x_cat1 == 'A'] += 3
y[x_cat1 == 'B'] -= 2

# Generate ratio metric: numerator/denominator, for example click/show
numerator_pre = y_pre 
numerator = y 
denominator_pre = x1 + 2*x5 + 3*np.log(x_long_tail1+1) + np.random.normal(0, 1, n)
denominator = denominator_pre + 2*treatment + treatment*(x1)*2 + np.random.normal(0, 2, n)
###################################################################################################### 


################################## generate observational data ####################################### 
# use x1,x2,x3 before
# Generate linear combination for t_ob
linear_combination_t = 1*x1 + 0.2*x2 + 0.5 * x3 + 0.1 * np.random.normal(0, 1, n)

# Convert the linear combination into a probability using the logistic function
prob_t = 1 / (1 + np.exp(-linear_combination_t))

# Generate binary variable t based on the probability
t_ob = np.random.binomial(1, prob_t)

# Generate linear combination for y
linear_combination_y = 0.5 * x1 - 0.25 * x2 + 0.1 * x3 + 0.5 * t_ob + 0.1 * np.random.normal(0, 1, n)

# Generate target variable y
y_ob = linear_combination_y + np.random.normal(0, 1, n)
###################################################################################################### 



# get DataFrame
df = pd.DataFrame({
    'x1': x1,
    'x2': x2,
    'x3': x3,
    'x4': x4,
    'x5': x5,
    'x_long_tail1': x_long_tail1,
    'x_long_tail2': x_long_tail2,
    'x_cat1': x_cat1,
    'treatment': treatment,
    't_ob':t_ob,
    'y': y,
    'y_ob': y_ob,
    'numerator_pre':numerator_pre,
    'numerator':numerator,
    'denominator_pre':denominator_pre,
    'denominator':denominator,
    'weight' : weight
})


# show data 
print(df.head())

Covariables

- `x1`: This variable follows a normal distribution with a mean of 0 and a variance of 1.
- `x2`: This variable also follows a normal distribution, but with a mean of 0 and a larger variance of 4.
- `x3`: This variable is generated from an exponential distribution with a rate parameter of 1, resulting in a variance of 1.
- `x4`: Similar to `x3`, this variable is generated from an exponential distribution, but with a larger rate parameter of 2, resulting in a variance of 4.
- `x5`: This variable is generated from a uniform distribution between 0 and 5, with an approximate variance of 1.33.
- `weight`: This variable is generated from a uniform distribution between 0 and 1 and can be used for sample reweighing.
- `x_long_tail1`: This variable follows a Pareto distribution with a shape parameter of 3, representing a long-tailed distribution.
- `x_long_tail2`: Similarly, this variable follows a Pareto distribution with a shape parameter of 2.
- `x_cat1`: This variable is a discrete variable of string type, randomly chosen from the categories 'A', 'B', 'C', 'D', and 'E'.

Experimental data
- `treatment`: This variable represents the treatment assignment and is a binary random variable.

- `y`: The target variable `y` is highly correlated with `x1`, `x2`, `x3`, `x_long_tail2`, and `treatment`. There is also heterogeneity in the relationship with `x_cat1` and `x1`, `x2`, `x4`. It is generated by adding `y_pre` with treatment effects, interaction terms, and random noise.

- `numerator_pre`: This variable is a precursor to the numerator of a ratio metric and is highly correlated with `y_pre`.

- `numerator`: The numerator of the ratio metric is derived from `numerator_pre` and is highly correlated with `y`.

- `denominator_pre`: This variable is a precursor to the denominator of a ratio metric and is generated based on `x1`, `x5`, and `x_long_tail1`.

- `denominator`: The denominator of the ratio metric is derived from `denominator_pre` and is influenced by treatment effects, interaction terms with `x1`, and random noise.

Observational data 
- `t_ob`: This binary variable is generated based on a linear combination of `x1`, `x2`, and `x3`. The linear combination is converted into a probability using the logistic function, and `t_ob` is generated by sampling from a binomial distribution with the probability.

- `y_ob`: The target variable `y_ob` is generated based on a linear combination of `x1`, `x2`, `x3`, and `t_ob`, along with random noise. It represents the outcome variable in the observational data.


### data description

In [ ]:
df = ais.readClickHouse('test_data_small')
df.dtypes

In [ ]:
df.head(2)

In [ ]:
# data descirption
df.describe('*')

#### histplot

In [ ]:
# histplot

from fast_causal_inference.lib.tools import *
col = 'x1'
histplot('test_data_small',col,bin_num=50)

col = 'x_cat1'
histplot('test_data_small',col,bin_num=50)

#### boxplot

In [ ]:
# boxplot

from fast_causal_inference.lib.tools import *

col = 'x_long_tail1'
boxplot('test_data_small',col)

# AB experiment Analysis

### ttest

- In A/B test analysis, **t-test** is widely used to test whether the average of treatment variant is statistically significantly different from the average of the control variant. However, the mean and variance formula only applied to i.i.d (independent and identically distributed) random variables, and in real business cases our metrics are more complex. Mostly, business metrics are defined as ratios, for example, Clickthrough rate or CTR which is defined as Clicks/Views. Here, we will *utilize the delta method to approximate the variance of the metrics ratio*.
$$t=\frac{\bar X_B-\bar X_A}{\sqrt{Var(\bar X_A)+Var(\bar X_B)}}$$

$$Var(\bar X)=\frac{1}{n}\frac{1}{n-1}\sum_{i=1}^n (X_i-\bar X)^2 $$
- **Delta method** extends the normal approximations of the central limit theorem. Delta method approximates asymptotically normal random variables by applying the Taylor series on the function of random variables. We can estimate the variance of x/y as follows:
$$v=(1/y,-x/y^2)|_{x=\bar x, y=\bar y )}$$

$$\mathop{{M}}\nolimits_{{2 \times 2}}=\frac{1}{n}{ \left[ {\begin{array}{*{20}{c}}
{var(x)}&{cov(x,y)}\\
{cov(x,y)}&{var(y)}\\
\end{array}} \right] }={ \left[ {\mathop{{m}}\nolimits_{{ij}}} \right] }  $$

$$var(x/y)=v*\mathop{{M}}\nolimits_{{2 \times 2}}*v$$

- <font size="4">case1: use deltamethod to do t-test</font>

In [ ]:
df.ttest_2samp('avg(numerator)/avg(denominator)','treatment','two-sided')

- <font size="4">case2: use deltamethod to do t-test and **CUPED** to reduce variance</font>  


    - We offer a feature in ttest_2samp that utilizes [CUPED (Controlled-experiment Using Pre-Experiment Data)](https://ai.stanford.edu/~ronnyk/2013-02CUPEDImprovingSensitivityOfControlledExperiments.pdf) This technique adjusts metrics using pre-experiment data from both control and treatment groups, aiming to decrease metric variability.
    - To further reduce variance, you can combine multiple metrics using the '+' operator.
    - Additionally, ttest_2samp allows you to perform a t-test grouped by any dimensions of your choice.

In [ ]:
df.ttest_2samp('avg(numerator)/avg(denominator)','treatment','two-sided',X='avg(numerator_pre)/avg(denominator_pre)+avg(x1)')

In [ ]:
df.groupBy('x_cat1').ttest_2samp('avg(numerator)/avg(denominator)','treatment','two-sided',X='avg(numerator_pre)/avg(denominator_pre)')

- <font size="4">case3: ttest_1samp </font>  
    - ttest_1samp can be used in pair test.


In [ ]:
df.ttest_1samp('avg(numerator)/avg(denominator)','two-sided')

### MWU test

The [Mann–Whitney U test](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) is a nonparametric test of the null hypothesis that, for randomly selected values X and Y from two populations, the probability of X being greater than Y is equal to the probability of Y being greater than X.

In [ ]:
df.mann_whitney_utest('numerator', 'treatment')

### SRM

**SRM (Sample ratio mismatch)** is an experimental flaw where the expected traffic allocation doesn’t fit with the observed visitor number for each testing variation. We do this using the chi-squared test of independence.

In [ ]:
df.srm('numerator', 'treatment', [1,1])

# Regression-based model

### Linear regression

In statistics, [linear regression](https://en.wikipedia.org/wiki/Linear_regression) is a linear approach for modelling the relationship between a scalar response and one or more explanatory variables (also known as dependent and independent variables). The case of one explanatory variable is called simple linear regression. Written in matrix notation as:
$$y=X\beta+\epsilon$$

#### OLS

In statistics, [ordinary least squares (OLS)](https://en.wikipedia.org/wiki/Ordinary_least_squares) is a type of linear least squares method for choosing the unknown parameters in a linear regression model (with fixed level-one effects of a linear function of a set of explanatory variables) by the principle of least squares: minimizing the sum of the squares of the differences between the observed dependent variable (values of the variable being observed) in the input dataset and the output of the (linear) function of the independent variable.
$$S(\boldsymbol{\beta}) = \sum_{i=1}^n \left| y_i - \sum_{j=1}^p X_{ij}\beta_j\right|^2 = \left\|\mathbf y - \mathbf{X} \boldsymbol \beta \right\|^2.$$
We can leverage **matrix multiplication** to compute Ordinary Least Squares (OLS), which is highly efficient in the OLAP engine we are using.
$$\hat{\boldsymbol{\beta}} = \left( \mathbf{X}^{\operatorname{T}} \mathbf{X} \right)^{-1} \mathbf{X}^{\operatorname{T}} \mathbf y.$$


In [ ]:
# ols
df.ols('y~x1+x2+x3+weight', use_bias=True).show()

In [ ]:
import fast_causal_inference.dataframe.regression as Regression
table = 'test_data_small'
df = fast_causal_inference.readClickHouse(table)
model = Regression.Ols(True)
model.fit('y~x1+x2+t_ob', df)
model.summary()
effect_df = model.effect('x1+x2+x3', df)
effect_df.show()

#### WLS

[Weighted least squares (WLS)](https://en.wikipedia.org/wiki/Weighted_least_squares), also known as weighted linear regression,[1][2] is a generalization of ordinary least squares and linear regression in which knowledge of the unequal variance of observations (heteroscedasticity) is incorporated into the regression. WLS is also a specialization of generalized least squares, when all the off-diagonal entries of the covariance matrix of the errors, are null.
$$  \underset{\boldsymbol\beta}{\operatorname{arg\ min}}\, \sum_{i=1}^{n} w_i \left|y_i - \sum_{j=1}^{m} X_{ij}\beta_j\right|^2 =
  \underset{\boldsymbol\beta}{\operatorname{arg\ min}}\, \left\|W^\frac{1}{2}\left(\mathbf{y} - X\boldsymbol\beta\right)\right\|^2.$$
We can leverage **matrix multiplication** to compute Weighted least squares (WLS), which is highly efficient in the OLAP engine we are using.
$$\hat{\boldsymbol{\beta}} = (X^\textsf{T} W X)^{-1} X^\textsf{T} W \mathbf{y}.$$

In [ ]:
import fast_causal_inference.dataframe.regression as Regression
model = Regression.Wls(weight='1', use_bias=True)
model.fit('y~x1+x2+x3', df)
model.summary()
effect_df = model.effect('x1+x2+x3', df)
effect_df.show()

#### Lasso

**Lasso (Least Absolute Shrinkage and Selection Operator)** is a method used in regression analysis that performs both variable selection and regularization. This enhances the prediction accuracy and interpretability of the statistical model it produces. Lasso introduces a penalty term to the loss function of the least square method, which is the absolute value of the magnitude of the coefficients. This results in some coefficients being shrunk to zero, effectively selecting a simpler model that does not include those coefficients. To estimate Lasso, we use **gradient descent**, an optimization algorithm.


In [ ]:
import fast_causal_inference.dataframe.regression as Regression
df.stochastic_logistic_regression('y~x1+x2+x3', learning_rate=0.00001, l1=0.1, batch_size=15, method='Lasso').show()
df.agg(Regression.stochastic_logistic_regression('y~x1+x2+x3', learning_rate=0.00001, l1=0.1, batch_size=15, method='SGD')).show()


### logistic model

In statistics, the [logistic model (or logit model)](https://en.wikipedia.org/w/index.php?title=Logistic_regression) is a statistical model that models the probability of an event taking place by having the log-odds for the event be a linear combination of one or more independent variables. In regression analysis, logistic regression (or logit regression) is estimating the parameters of a logistic model (the coefficients in the linear combination). To estimate logistic model, we use **gradient descent**, an optimization algorithm.

$$p(x)=\frac{1}{1+e^{-(\beta_0+\beta_1 x)}}$$

In [ ]:
import fast_causal_inference
from fast_causal_inference.dataframe.regression import Logistic
table = 'test_data_small'
df = fast_causal_inference.readClickHouse(table)
X = ['x1', 'x2', 'x3', 'x4', 'x5', 'x_long_tail1', 'x_long_tail2']
Y = 't_ob'
logit = Logistic(tol=1e-6, iter=500)
logit.fit(Y, X, df)
logit.summary()

### IV


[instrumental variables (IV)](https://en.wikipedia.org/wiki/Instrumental_variables_estimation) is a method used in statistics, econometrics, epidemiology, and related disciplines to estimate causal relationships when controlled experiments are not feasible or when a treatment is not successfully delivered to every unit in a randomized experiment. 

The idea behind IV is to use a variable, known as an instrument, that is correlated with the endogenous explanatory variables (the variables that are correlated with the error term), but uncorrelated with the error term itself. This allows us to isolate the variation in the explanatory variable that is purely due to the instrument and thus uncorrelated with the error term, which can then be used to estimate the causal effect of the explanatory variable on the dependent variable.

Here is an example:

1. $t_1 = treatment + X_1 + X_2$
2. $Y = \hat t_1 + X_1 + X_2$

- $X_1$ and $X_2$ are independent variables or predictors.
- $t_1$ is the dependent variable that you are trying to explain or predict.  
- $treatment$ is an independent variable representing some intervention or condition that you believe affects $t_1$. 
- $Y$ is the dependent variable that you are trying to explain or predict
-  $\hat t_1$ is the predicted value of $t_1 from the first equation


We first regress $X_3$ on the treatment and the other exogenous variables $X_1$ and $X_2$ to get the predicted values $\hat t_1$. Then, we replace $t_1$ with $\hat t_1$ in the second equation and estimate the parameters. This gives us the causal effect of $t_1$ on $Y$, purged of the endogeneity problem.

In [ ]:
import fast_causal_inference.dataframe.regression as Regression
model = Regression.IV()
model.fit(df,formula='y~(t_ob~treatment)+x1+x2')
model.summary()

df.iv_regression('y~(t_ob~treatment)+x1+x2').show()
df.agg(Regression.iv_regression('y~(t_ob~treatment)+x1+x2')).show()

# Obeservational Analysis

### Case1- PSM

[Propensity score matching (PSM)](https://en.wikipedia.org/wiki/Propensity_score_matching) is a quasi-experimental method in which the researcher uses statistical techniques to construct an artificial control group by matching each treated unit with a non-treated unit of similar characteristics. Using these matches, the researcher can estimate the impact of an intervention.
![Example Image](https://builtin.com/sites/www.builtin.com/files/styles/ckeditor_optimize/public/inline-images/1_propensity-score-matching.jpeg)

#### predict propensity score

Once we have collected the data, we can build the propensity model predicting the probability of receiving the treatment given the confounders. Typically, logistic regression is used for this classification model. Let’s build a propensity model:

In [ ]:
import fast_causal_inference
Y='y'
T='t_ob'
table = 'test_data_small'
X = [ 'x1', 'x2', 'x3']

In [ ]:
import fast_causal_inference.dataframe.regression as Regression
model = Regression.StochasticLogisticRegression(learning_rate=0.00001, l1=0.1, batch_size=15, method='SGD')

df = ais.readClickHouse('test_data_small')
model.fit('y~x1+x2+x3', df)
effect_df = model.effect('x1+x2+x3', df)
effect_df

#### Matching

We will be performing one-to-one matching to find the most similar control records for each passenger in the treatment group. Matching based on the propensity score, which is a balancing score, allows us to ensure that the distribution of confounders between the matched records is likely to be similar.

In [ ]:
import time
table_match = f"{table}_{int(time.time())}_matched"
fast_causal_inference.clickhouse_create_view(clickhouse_view_name=table_match, sql_statement=f"""
select *,caliperMatching(if({T}=1,1,-1),effect,0.05) AS matchingIndex 
from {effect_df.getTableName()} where matchingIndex!=0 """, primary_column="matchingIndex",is_force_materialize=True, is_sql_complete=True, is_use_local=True)
match_df = ais.readClickHouse(table_match)
match_df

#### Balance check

It’s time to evaluate how good the matching was. Let’s inspect if the groups look more comparable in terms of the confounders:

In [ ]:
# before matched
from fast_causal_inference.lib.tools import *
SMD(effect_df.getTableName(),T,X)

In [ ]:
from fast_causal_inference.lib.tools import *
matching_plot(effect_df.getTableName(),T,'effect')

In [ ]:
# After matched
from fast_causal_inference.lib.tools import *
SMD(match_df.getTableName(),T,X)

In [ ]:
from fast_causal_inference.lib.tools import *
matching_plot(match_df.getTableName(),T,'effect')

#### Evaluate treatment effect on the outcome

Now, it’s time to familiarize ourselves with a few terms related to the treatment effect, also known as the causal effect. Looking at a small example with a continuous outcome may be the easiest way to get familiarized with it.
##### DIM estimator
After the matching process, we can consider the experimental and control groups as homogeneous, free from bias. This allows us to directly estimate the Average Treatment Effect (ATE) and compute its variance using the bootstrap method. The `ATEestimator` function also returns the confidence interval for the DIM(Difference in Means) estimator. As per the formula below:

- $\hat \tau_{ipw}$ represents the ATE estimator. 

For each sample $i=1,...,N$:

- $Y_i$ is the outcome variable
- $Z_i$ represents the treatment
- $X_i$ is the covariate.

$$\hat \tau_{ATE}=\frac{\sum_{i=1}^N Y_iZ_i}{\sum_{i=1}^NZ_i}-\frac{\sum_{i=1}^N Y_i(1-Z_i))}{\sum_{i=1}^N(1-Z_i)}$$

##### IPW estimator
Without resorting to matching, we can directly estimate the Average Treatment Effect (ATE) using all available samples. This process involves the use of the Inverse Probability Weighting (IPW) estimator for ATE estimation, with the variance calculated via the bootstrap method. The `IPWestimator` function also yields the confidence interval of the ATE. As illustrated in the formula below:

- $\hat \tau_{ipw}$ is the IPW estimator. 

For each sample $i=1,...,N$:

- $Y_i$ is the outcome variable
- $Z_i$ is the treatment
- $X_i$ is the covariate
- $e(X_i)$ is the predicted propensity score.

$$\hat \tau_{ipw}=\frac{\sum_{i=1}^N Y_iZ_i/e(X_i)}{\sum_{i=1}^NZ_i/e(X_i)}-\frac{\sum_{i=1}^N Y_i(1-Z_i)/(1-e(X_i))}{\sum_{i=1}^N(1-Z_i)/(1-e(X_i))}$$

### Case2- Uplift modeling

Uplift modeling is a set of causal inference techniques that leverage machine learning models to estimate the causal impact of a treatment on an individual's behavior.

In this context:

- "Persuadables" are individuals who respond positively to the treatment.
- "Sleeping dogs" are individuals who exhibit a strong negative response to the treatment.
- "Lost causes" are individuals who do not reach the desired outcome even with the treatment.
- "Sure things" are individuals who always achieve the desired outcome, regardless of the treatment.

The objective is to identify the "persuadables" to focus efforts on, avoid wasting resources on "sure things" and "lost causes," and prevent unnecessary intervention for "sleeping dogs."

![Example Image](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSsUVyoXY2SaSAdDVAIur1ssPPU8NZ0Qwlff252bgd8zVDOye08yErMf-aIZy_hNOMk_RY&usqp=CAU)

#### data split

In [ ]:
table = 'test_data_small'
Y='y'
T='treatment'
X = 'x1+x2+x3+x4+x5+x_long_tail1+x_long_tail2'
needcut_X = 'x1+x2+x3+x4+x5+x_long_tail1+x_long_tail2'

from fast_causal_inference.lib.tools import *
table_train,table_test = data_split(table)

#### causal tree

The causal tree is a method used in [Recursive Partitioning for Heterogeneous Causal Effects](https://www.pnas.org/doi/10.1073/pnas.1510489113) to estimate the individual treatment effects within a population. It is a variant of traditional decision trees that aims to identify subgroups of individuals who respond differently to a treatment.

The causal tree algorithm recursively partitions the data based on a set of covariates and their treatment assignment. It seeks to find the optimal splits that maximize the heterogeneity in treatment effects across the resulting subgroups. This allows for the identification of subpopulations that exhibit varying responses to the treatment.

The causal tree approach is valuable in understanding and predicting individual treatment effects in situations where the treatment effect may vary across different subpopulations. It provides a useful tool for personalized decision-making and targeted interventions based on the identified subgroups with distinct treatment responses.

##### train

In [ ]:
from fast_causal_inference.lib.causaltree import CausalTree
hte = CausalTree(depth = 3,min_sample_ratio_leaf=0.001)
hte.fit(Y,T,X,needcut_X,table_train)

##### tree visualization

In [ ]:
treeplot = hte.treeplot()
treeplot.render('digraph.gv', view=False) # save to file digraph.gv.pdf
treeplot

In [ ]:
# uplift curve in training data
hte.hte_plot() 

#### Model Evaluation

Since actual uplift can't be observed for each individual, measure the uplift over a group of customers.

Uplift Curve: plots the real cumulative uplift across the population
- First, rank the test dataframe order by the predict uplift.
- Next, calculate the cumulative percentage of visits in each group (treatment or control).
- Finally, calculate the group's uplift at each percentage.


In [ ]:

from fast_causal_inference.lib.causaltree import CausalTree
from fast_causal_inference.lib.metrics import *

table_test_predict = f'{table_test}_pred'
# clickhouse_drop_view(clickhouse_view_name=table_test_predict) # drop table
hte.effect_2_clickhouse(table_test_predict,
                        table_input=table_test,
                        keep_col='*')
tmp1 = get_lift_gain("effect", Y, T, table_test_predict,discrete_treatment=True, K=100)
print(tmp1)
hte_plot([tmp1])